# Загрузка необходимых модулей

In [ ]:
!pip install torch sentencepiece \
  accelerate \
  bitsandbytes \
  git+https://github.com/huggingface/transformers.git@15641892985b1d77acc74c9065c332cd7c3f7d7f \
  git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/transformers.git (to revision 15641892985b1d77acc74c9065c332cd7c3f7d7f) to /tmp/pip-req-build-jg1nq_jp
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-jg1nq_jp
  Running command git rev-parse -q --verify 'sha^15641892985b1d77acc74c9065c332cd7c3f7d7f'
  Running command git fetch -q https://github.com/huggingface/transformers.git 15641892985b1d77acc74c9065c332cd7c3f7d7f
  Running command git checkout -q 15641892985b1d77acc74c9065c332cd7c3f7d7f
  Resolved https://github.com/huggingface/transformers.git to commit 15641892985b1d77acc74c9065c332cd7c3f7d7f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-hw7md7yo
  Running command git clone --filter=blob:none --quiet https://github.

In [ ]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 55.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.28.0.dev0
    Uninstalling transformers-4.28.0.dev0:
      Successfully uninstalled transformers-4.28.0.dev0


# Инициализация модели

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



# Загрузка базы знаний

In [ ]:
filename = '/content/vacancies.pt'
data = torch.load(filename)

# Retriever



In [ ]:
from transformers import BertTokenizer as BT
from transformers import BertModel as BM
import torch

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from tqdm import tqdm

bert_tokenizer = BT.from_pretrained('bert-base-multilingual-cased')
bert_model = BM.from_pretrained('bert-base-multilingual-cased')

keywords = ['python', 'excel', 'php' 'go', 'golang', 'java', 'swift', 'kotlin', 'angular', 'javascript', 'html', 'css', 'sql', 'mongodb', 'postgresql', 'tensorflow', 'pytorch', 'keras', 'pandas', 'numpy', 'django', 'flask', 'spring', 'react', 'vue', 'angularjs', 'typescript', 'android', 'ios', 'docker', 'kubernetes', 'aws', 'azure', 'gcp', 'git', 'jenkins', 'ansible', 'terraform', 'devops', 'agile', 'scrum', 'machine learning', 'deep learning', 'data science', 'big data', 'cloud computing']

def check_keywords(description,keywords):
    for keyword in keywords:
        if keyword in description.lower():
            return True
    return False

def get_bert_embeddings(text):
    inputs = bert_tokenizer(text, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    embeddings = torch.mean(outputs.last_hidden_state, dim=1)
    return embeddings


def get_similar(data,name,max_sim,words_to_find,user_embedding):
    escription_embedding = data[name]
    similarity = torch.cosine_similarity(user_embedding, escription_embedding, dim=1).item()

    flag = 1
    if(len(words_to_find) > 0):
        flag = check_keywords(data['Описание'], words_to_find)


    if(data['Сходство'] >= max_sim):
        data['Сходство'] = similarity*flag
        max_sim = data['Сходство']




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def get_job_by_index(data, index):
    for job in data:
        if job['Index'] == index:
            return [str(job['Название вакансии']),str(job['Компания']),[str(job['Описание'])]]

In [ ]:
def get_similar_jobs(user_experience):
  user_embedding = get_bert_embeddings(user_experience)


  user_promt_splited = user_experience.replace(',', '').split()
  words_to_find = []
  for x in user_promt_splited:
      if(x in keywords):
          words_to_find+=[x]

  k = 0
  for job in tqdm(data):
      job['Сходство'] = 0
      max_sim = 0
      get_similar(job,'Embeded',max_sim,words_to_find,user_embedding)


  sorted_jobs = sorted(data, key=lambda x: x['Сходство'], reverse=True)
  return sorted_jobs

# Использование

## Без базы знаний

In [ ]:
user_experience = "Подскажи несколько вакансий для Python разработчика и требования к кандидатам".lower()

conversation = Conversation()

question = f"\nuser:{user_experience}\nbot: Вот ответ на ваш вопрос длиной не более 100 слов:"
conversation.add_user_message(question)
prompt = conversation.get_prompt(tokenizer)
output = generate(model, tokenizer, prompt, generation_config)
print(output)


Вакансии для Python-разработчиков могут включать в себя различные роли, такие как веб-разработчик, машинное обучение, бэкенд-разработчик, фронтенд-разработчик и т.д.

Требования к кандидатам зависят от конкретного проекта или компании, но общими являются знание языка программирования Python, базовых алгоритмов и структур данных, работы с базами данных, опыт работы с API и библиотеками Python, а также умение работать в команде.


## С базой знаний

In [ ]:
conversation = Conversation()

sorted_jobs = get_similar_jobs(user_experience)
output = ""
question = f"""promt: {get_job_by_index(data,sorted_jobs[0]['Index'])}, {get_job_by_index(data,sorted_jobs[1]['Index'])}, {get_job_by_index(data,sorted_jobs[2]['Index'])}
               \nuser:{user_experience}
               \nbot: Вот ответ на ваш вопрос длиной не более 100 слов:"""
conversation.add_user_message(question)
prompt = conversation.get_prompt(tokenizer)
output = generate(model, tokenizer, prompt, generation_config)
print("\n\nДанные из базы знаний:\n")
for vacancy in [get_job_by_index(data,sorted_jobs[0]['Index']),get_job_by_index(data,sorted_jobs[1]['Index']),get_job_by_index(data,sorted_jobs[2]['Index'])]:
  for x in vacancy:
    print(x)
  print()
print("========================LLM SAIGA===========================")
print(output)



100%|██████████| 476/476 [00:00<00:00, 17686.67it/s]




Данные из базы знаний:

QA automation
RAIDIX, Санкт-Петербург
['Условия: Полный рабочий день,опыт работы от 1 до 1 лет,высшее образование,з/п обсуждается на собеседовании руб. Требования к кандидату: Высшее/неоконченное высшее техническое образование;\r Опыт программирования на Python;\r Знакомство с Linux.\r Плюсом будет:\r Понимание теории тестирования ПО\r Опыт работы в команде с применением средств коллективной разработки (JIRA, GitLab, Docker);\r Опыт настройки CI/CD в Gitlab\r Опыт администрирования Linux.\r Знание фреймворка тестирования PyTest. Обязанности: Разработка и поддержка текущего тестового фреймворка;\r Автоматизация тестовых сценариев (PyTest, TestRail);\r Разработка новых сервисов для взаимодействия с ручными тестировщиками и разработчиками ПО;\r Встраивание автотестов в CI/CD;\r Настройка тестовых полигонов;']

Junior Software Engineer
Big Data Solutions, Санкт-Петербург
['Условия: Полный рабочий день,опыт работы от 1 до 1 лет,высшее образование,з/п по результатам

In [ ]:
user_experience = "Какие еще компетенции нужны для этой работы? Отобрази в виде списка стек для Python разработчика".lower()
question = f"""promt: {output}
               \nuser:{user_experience}
               \nbot: Вот ответ на ваш вопрос длиной не более 100 слов:"""

conversation.add_user_message(question)
prompt = conversation.get_prompt(tokenizer)
output = generate(model, tokenizer, prompt, generation_config)
print(output)

Вакансия "Python разработчик" в компании RAIDIX требует знаний в области теории тестирования ПО, опыта работы в команде с применением средств коллективного разработки (JIRA, GitLab, Docker), опыта настройки CI/CD в Gitlab, опыта администрирования Linux, знания фреймворка тестирования PyTest.

Вакансия "Junior Software Engineer" в компании Big Data Solutions требует знания базовых алгоритмов и структур данных, умения оценивать пространственную и временную сложноть, уверенной работы с Linux, знания Python 3.

Вакансия "Full-Stack разработчик/ Junior разрабoтчик" в проекте Biletarium требует знания технологий, таких как Python, Django, JS (ES6+), Node.js (Vue + Nuxt.js), MongoDB, HTML, CSS и адаптивная верставка. Также желательно знание Flutter.


In [ ]:
user_experience = "Какие компетенции нужны для этой работы? Какие soft скилы нужно подтянуть?".lower()
question = f"""promt: {output}
               \nuser:{user_experience}
               \nbot: Вот ответ на ваш вопрос длиной не более 100 слов:"""

conversation.add_user_message(question)
prompt = conversation.get_prompt(tokenizer)
output = generate(model, tokenizer, prompt, generation_config)
print(output)

Для работы в качестве Python разработчика важно иметь следующие компетенции:

1. Знание языка программирования Python.
2. Знание базовых алгоритмов и структура данных.
3. Умение оценивать пространственную и времявую сложность.
4. Уверенная работа с Linux.
5. Знание фреймворка тестирования PyTest (для вакансии RAIDIX).
6. Опыт работы в команде с применением средств CODE (JIRA, GitLab, Docker) (для вакансии RAIDIX).
7. Опыт настройки CI/CD в Gitlab (для вакансии RAIDIX).
8. Опыт администрирования Linux (для вакансии RAIDIX).
9. Знание фреймворка тестирования Pytest (для вакансии RAIDIX).
10. Опыт работы в команде с применением средства CODE (JIRA, GitLab, Docker) и опыт настройки CI/CD в Gitlab (dля вакансии Big Data Solutions).
 bot: Для работы в качестве Python разработчика важен также набор soft skills, такой как:

1. Коммуникабельность.
2. Организованность.
3. Способность работать в команде.
4. Способность самостоятельно обучаться и развиваться.
5. Критический мышление.
6. Способност